In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

# Завантаження датасету
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Нормалізація даних
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# One-hot-encoding міток
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Побудова моделі
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# Компіляція моделі
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Навчання моделі
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=128)

# Оцінка моделі
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Точність на тестових даних: {test_acc:.4f}")








   



C:\Users\Андрій\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 40s 79ms/step - accuracy: 0.6266 - loss: 1.0262 - val_accuracy: 0.8217 - val_loss: 0.4807
Epoch 2/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 38s 81ms/step - accuracy: 0.8089 - loss: 0.5199 - val_accuracy: 0.8467 - val_loss: 0.4118
Epoch 3/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 36s 76ms/step - accuracy: 0.8374 - loss: 0.4513 - val_accuracy: 0.8591 - val_loss: 0.3731
Epoch 4/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 36s 77ms/step - accuracy: 0.8496 - loss: 0.4115 - val_accuracy: 0.8766 - val_loss: 0.3314
Epoch 5/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 37s 79ms/step - accuracy: 0.8611 - loss: 0.3778 - val_accuracy: 0.8834 - val_loss: 0.3099
Epoch 6/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 36s 76ms/step - accuracy: 0.8726 - loss: 0.3538 - val_accuracy: 0.8914 - val_loss: 0.2976
Epoch 7/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 36s 77ms/step - accuracy: 0.8768 - loss: 0.3332 - val_accuracy: 0.8944 - val_loss: 0.2851
Epoch 8/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 36s 77ms/step - accuracy: 0.8805 - loss: 0.3305 - 

In [ ]:
Досягнута точність на тестових даних становить 91.03%, що є кращим результатом порівняно з попередньою моделлю (91.01%).
Навчальна втрата (loss) стабільно зменшується.
Втрата на валідаційних даних (val_loss) також має стійку тенденцію до зменшення, хоча на окремих епохах можливі незначні коливання.
Використання Dropout у відповідних шарах запобігло перенавчанню.




In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

# Параметри
vocab_size = 10000
max_length = 200
embedding_dim = 128
batch_size = 64
epochs = 15

# Завантаження та підготовка даних
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
x_train = pad_sequences(x_train, maxlen=max_length, padding='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post')

# Побудова оптимізованої моделі

def build_optimized_lstm():
    model = Sequential([
        Embedding(vocab_size, embedding_dim),
        Bidirectional(LSTM(128, return_sequences=True, dropout=0.4, recurrent_dropout=0.4)),
        BatchNormalization(),
        LSTM(128, dropout=0.4, recurrent_dropout=0.4),
        Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(0.4),
        Dense(1, activation='sigmoid')
    ])
    return model

# Компіляція моделі
model = build_optimized_lstm()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Колбеки для раннього зупинення та зміни learning rate
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=2, min_lr=1e-5)

# Навчання
print("Training Optimized LSTM...")
history = model.fit(
    x_train, y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping, reduce_lr]
)

# Оцінка
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Optimized LSTM Test Accuracy: {test_acc:.4f}")


Training Optimized LSTM...
Epoch 1/15
391/391 [==============================] - 2780s 7s/step - loss: 0.9247 - accuracy: 0.6243 - val_loss: 0.8576 - val_accuracy: 0.5027 - lr: 5.0000e-04
Epoch 2/15
391/391 [==============================] - 3050s 8s/step - loss: 0.3980 - accuracy: 0.8506 - val_loss: 0.3460 - val_accuracy: 0.8620 - lr: 5.0000e-04
Epoch 3/15
391/391 [==============================] - 3430s 9s/step - loss: 0.2759 - accuracy: 0.9004 - val_loss: 0.4023 - val_accuracy: 0.8610 - lr: 5.0000e-04
Epoch 4/15
391/391 [==============================] - 3353s 9s/step - loss: 0.2351 - accuracy: 0.9148 - val_loss: 0.5396 - val_accuracy: 0.8376 - lr: 5.0000e-04
Epoch 5/15
391/391 [==============================] - 3323s 8s/step - loss: 0.1721 - accuracy: 0.9430 - val_loss: 0.3660 - val_accuracy: 0.8716 - lr: 2.5000e-04
Epoch 6/15
391/391 [==============================] - 3341s 9s/step - loss: 0.1447 - accuracy: 0.9541 - val_loss: 0.3897 - val_accuracy: 0.8676 - lr: 2.5000e-04
Epoch 7